# Lecture 10. Character-level Text Generation

## 10.1 Text 처리 
- 텍스트 데이터는 가장 흔한 시퀀스 형태의 데이터
- Document classification, sentiment analysis, Question answering 등에 활용 
- 텍스트 원본을 모형의 입력값으로 사용하지 못하기 때문에 텍스트를 수치형 텐서로 변환시키는 과정이 필요함

- 텍스트 $\rightarrow$ 토큰 $\rightarrow$ 벡터 
![](figures/token.PNG)

### Tokenization
        
- 텍스트를 수치형 텐서로 변환하기 위해 나누는 단위
    - 각 단어를 하나의 벡터로 변환
            {“the”, “cat”, “sat”, “on”, “the”, “mat”, “.”}
    - 각 문자를 하나의 벡터로 변환
            {"a","c","e","h","m","n","o","s","t","."}
    - n-gram(연속된 단어나 문자의 그룹)을 추출하여 벡터로 변환 
        - 2-grams
                {"The", "The cat", "cat", "cat sat", "sat", "sat on", "on", "on the", "the", "the mat", "mat"}
        - 3-grams
                {"The", "The cat", "cat", "cat sat", "The cat sat", "sat", "sat on", "on", "cat sat on", "on the", "the", "sat on the", "the mat", "mat", "on the mat"}

- 토큰의 집합을 vocabulary, dictionary 라고 일컬음

### 단어와 문자의 인코딩
1. One-hot encoding
    - 토큰을 벡터화 하는 가장 기본적인 방법
    - 모든 단어에 고유한 정수 인덱스를 부여
    - 정수 인덱스를 vocabulary size 크기의 binary 벡터로 변환 

In [1]:
# Toy example
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
text = " ".join(samples)
text

'The cat sat on the mat. The dog ate my homework.'

In [2]:
chars = set([c for c in text])
chars

{' ',
 '.',
 'T',
 'a',
 'c',
 'd',
 'e',
 'g',
 'h',
 'k',
 'm',
 'n',
 'o',
 'r',
 's',
 't',
 'w',
 'y'}

In [3]:
nb_chars = len(chars)
print(nb_chars)

18


In [4]:
char2index = dict((c, i) for i, c in enumerate(sorted(chars)))
print(char2index)

{' ': 0, '.': 1, 'T': 2, 'a': 3, 'c': 4, 'd': 5, 'e': 6, 'g': 7, 'h': 8, 'k': 9, 'm': 10, 'n': 11, 'o': 12, 'r': 13, 's': 14, 't': 15, 'w': 16, 'y': 17}


In [5]:
char2index = dict((c, i) for i, c in enumerate(sorted(chars)))
index2char = dict((i, c) for i, c in enumerate(sorted(chars)))
print(index2char)

{0: ' ', 1: '.', 2: 'T', 3: 'a', 4: 'c', 5: 'd', 6: 'e', 7: 'g', 8: 'h', 9: 'k', 10: 'm', 11: 'n', 12: 'o', 13: 'r', 14: 's', 15: 't', 16: 'w', 17: 'y'}


In [6]:
import numpy as np

max_length = 50
results = np.zeros((len(samples), max_length, len(char2index)))

In [7]:
print(len(samples))
print(max_length)
print(len(char2index))

2
50
18


In [8]:
# 원핫인코딩
import numpy as np

max_length = 50
results = np.zeros((len(samples), max_length, len(char2index)))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
        results[i, j, char2index[character]] = 1.

results        

array([[[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

2. Word embedding
    - One-hot encoding을 하게 되면 한 단어를 나타내는 벡터의 길이가 vocabulary size와 같기 때문에 일반적으로 매우 고차원이고 대부분이 0으로 채워져 있음: 비효율적
    - 0 또는 1로 채워진 고차원의 벡터 대신 실수값으로 채워져 있는 저차원 벡터로 표현하는 방법을 사용 
    - 저차원에 더 많은 정보를 저장할 수 있어 효율적임
    - Lecture 11에서 자세히 다룰 예정 

![](figures/onehot_embed.PNG)

## 10.2 Character-level text generation model
- 소설책의 텍스트를 학습하여 문장을 자동생성하는 모델
-  Alice's Adventures in Wonderland


      
    ... Alice was beginning to get very tired of sitting by her sister
    on the bank, and of having nothing to do:  once or twice she had
    peeped into the book her sister was reading, but it had no
    pictures or conversations in it, `and what is the use of a book,'
    thought Alice `without pictures or conversation?' ...
    
- 첫 10개의 문자를 입력하여 다음에 나타날 문자를 예측

<img src="figures/text_gen.png" width="40%" align="left">
<img src="figures/text_gen2.PNG" width="30%">



### Preparing the data

- 텍스트를 한 줄씩 불러들여 소문자 변환, 빈 줄 삭제 등의 기본적인 전처리를 진행

In [9]:
# -*- coding: utf-8 -*-
INPUT_FILE = "./data/alice_in_wonderland.txt"

fin = open(INPUT_FILE, 'rb') # 바이너리 파일을 읽기 모드로 오픈

lines = []
i=0
for line in fin: # 파일을 한 줄씩 읽어들임
    line = line.strip().lower() # 공백을 제거하고 소문자로 변환
    line = line.decode("ascii") # 디코딩하여 char로 변환
    if len(line) == 0: # 빈 줄 삭제
        continue
    lines.append(line)
fin.close()

text = " ".join(lines) # 모든 줄을 하나로 이어붙임

- 문자 수준의 One-hot encoding을 하기 위해 유일한 문자들의 집합인 vocabulary 생성

In [10]:
chars = set([c for c in text])
nb_chars = len(chars)

In [11]:
nb_chars

45

* char와 index를 연결하는 lookup table 구축


In [12]:
char2index = dict((c, i) for i, c in enumerate(sorted(chars)))
index2char = dict((i, c) for i, c in enumerate(sorted(chars)))

In [13]:
char2index

{' ': 0,
 '!': 1,
 '"': 2,
 "'": 3,
 '(': 4,
 ')': 5,
 '*': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '3': 11,
 ':': 12,
 ';': 13,
 '?': 14,
 '[': 15,
 ']': 16,
 '_': 17,
 '`': 18,
 'a': 19,
 'b': 20,
 'c': 21,
 'd': 22,
 'e': 23,
 'f': 24,
 'g': 25,
 'h': 26,
 'i': 27,
 'j': 28,
 'k': 29,
 'l': 30,
 'm': 31,
 'n': 32,
 'o': 33,
 'p': 34,
 'q': 35,
 'r': 36,
 's': 37,
 't': 38,
 'u': 39,
 'v': 40,
 'w': 41,
 'x': 42,
 'y': 43,
 'z': 44}

In [14]:
char2index['a']

19

In [15]:
index2char

{0: ' ',
 1: '!',
 2: '"',
 3: "'",
 4: '(',
 5: ')',
 6: '*',
 7: ',',
 8: '-',
 9: '.',
 10: '0',
 11: '3',
 12: ':',
 13: ';',
 14: '?',
 15: '[',
 16: ']',
 17: '_',
 18: '`',
 19: 'a',
 20: 'b',
 21: 'c',
 22: 'd',
 23: 'e',
 24: 'f',
 25: 'g',
 26: 'h',
 27: 'i',
 28: 'j',
 29: 'k',
 30: 'l',
 31: 'm',
 32: 'n',
 33: 'o',
 34: 'p',
 35: 'q',
 36: 'r',
 37: 's',
 38: 't',
 39: 'u',
 40: 'v',
 41: 'w',
 42: 'x',
 43: 'y',
 44: 'z'}

In [16]:
index2char[12]

':'

- Input sequence와 output label 생성 
- SEQLEN: 다음 문자를 예측하기 위해 입력할 문자의 수 
- STEP: 몇 개씩 건너뛰며 window를 이동할 것인가? 
- Ex: Input text= "The sky was falling"
    - The sky wa -> s
    - he sky was -> " "  
    - e sky was  -> f
    - sky was f -> a
    - sky was fa -> l

In [17]:
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [18]:
print(input_chars[0])

alice's ad


In [19]:
print(label_chars[0])

v


In [20]:
input_chars[0:10],label_chars[0:10]

(["alice's ad",
  "lice's adv",
  "ice's adve",
  "ce's adven",
  "e's advent",
  "'s adventu",
  's adventur',
  ' adventure',
  'adventures',
  'dventures '],
 ['v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ', 'i'])

In [21]:
len(input_chars)

143504

- 위에서 만든 input/output 문자 셋을 one-hot encoding으로 변환하여 모형에 입력 가능한 형태로 변환

In [22]:
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [23]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False])

- input과 output chars를 nb_chars 길이의 one-hot vector로 표현 
- input
    - (len(input_chars), SEQLEN, nb_chars)
    - SEQLEN 개 시점의 nb_chars 차원의 벡터가 input shape
- output
    - (len(input_chars), nb_chars)
    - (SEQLEN, nb_chars) 차원의 각 input에 대응하는 output label

In [24]:
print(input_chars[0])
print(X[0].shape)
print(y.shape)

alice's ad
(10, 45)
(143504, 45)



### Building the network

In [25]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras import optimizers
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard 
import time 

HIDDEN_SIZE = 32
BATCH_SIZE = 128

K.clear_session()
model = Sequential()
model.add(LSTM(HIDDEN_SIZE, return_sequences=True, input_shape=(SEQLEN, nb_chars), activation='relu'))
model.add(LSTM(HIDDEN_SIZE, return_sequences=False, activation='relu'))
model.add(Dense(HIDDEN_SIZE, activation='relu'))
model.add(Dense(nb_chars, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=optimizers.adam(lr=0.001))

#now = time.strftime("%c")
now = time.strftime('%Y-%m-%d', time.localtime(time.time()))
callbacks_list = [
    #ModelCheckpoint(filepath='models/text_gen.h5', monitor='val_loss', save_best_only=True),
    TensorBoard(log_dir='log/log'+now),
    EarlyStopping(monitor='val_loss',patience=3)
]
model.fit(X, y, batch_size=BATCH_SIZE, epochs=10, validation_split=0.2, callbacks=callbacks_list)

Using TensorFlow backend.


Train on 114803 samples, validate on 28701 samples
Epoch 1/10
  1536/114803 [..............................] - ETA: 1:53 - loss: 3.7904

C:\Users\028\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:92: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.224731). Check your callbacks.
  warnings.warn(


114803/114803 [==============================] - 11s 95us/step - loss: 2.7952 - val_loss: 2.4149
Epoch 2/10
114803/114803 [==============================] - 9s 82us/step - loss: 2.2226 - val_loss: 2.1693
Epoch 3/10
114803/114803 [==============================] - 9s 82us/step - loss: 2.0833 - val_loss: 2.0702
Epoch 4/10
114803/114803 [==============================] - 9s 82us/step - loss: 2.0120 - val_loss: 2.0121
Epoch 5/10
114803/114803 [==============================] - 9s 82us/step - loss: 1.9578 - val_loss: 1.9850
Epoch 6/10
114803/114803 [==============================] - 9s 81us/step - loss: 1.9170 - val_loss: 1.9493
Epoch 7/10
114803/114803 [==============================] - 10s 83us/step - loss: 1.8781 - val_loss: 1.9066
Epoch 8/10
114803/114803 [==============================] - 9s 83us/step - loss: 1.8457 - val_loss: 1.8945
Epoch 9/10
114803/114803 [==============================] - 10s 84us/step - loss: 1.8195 - val_loss: 1.8598
Epoch 10/10
114803/114803 [==================


### Prediction

- random seed 선택 (10개의 char로 이루어진 부분 문장)
- 다음번 문자로 나타날 확률이 가장 높은 문자 프린트 (ypred)
- "앞에서 사용된 9개 문자 + 새로 발생된 문자 1개"를 input으로 사용 
- 반복을 통해 지정한 개수 만큼 문자 발생 
<img src="figures/text_gen_pred.png" width="50%">

- 문자열을 입력하여 그 다음에 나타날 확률일 가장 높은 문자를 무조건 발생시키면 비슷한 상황에서 언제나 같은 문자를 만들어냄
- 무조건 가장 높은 문자를 출력하는 대신 확률적으로 발생시킨다면 보다 다양한 문장을 생성할 수 있음 
    - e.g) {"a","b","c"}에 대한 output이 {0.2, 0.5, 0.3}인 경우 언제나 "b"를 출력하기 보다는 0.5의 확률로 "b"를 출력(multinomial distribution 활용)
- Output 값이 가장 큰 문자를 얼마나 "확실히" 출력할 것인가? temperature 값으로 조정하기 위해 `sample` 함수 사용

In [26]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    #print(preds.round(3))
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [28]:
sample([0.2, 0.5, 0.3], temperature=1)

2

In [29]:
model.save('text_gen.h5')

In [30]:
from keras.models import load_model
model=load_model('text_gen.h5')

In [31]:
test_idx = np.random.randint(len(input_chars))
test_chars = input_chars[test_idx]
print("Generating from seed: %s" % (test_chars))
print(test_chars, end="")
for i in range(400):
    Xtest = np.zeros((1, SEQLEN, nb_chars))
    for i, ch in enumerate(test_chars):
        Xtest[0, i, char2index[ch]] = 1
    pred = model.predict(Xtest, verbose=0)[0]
    ypred = index2char[sample(pred, 0.5)]
    print(ypred, end="")
    # move forward with test_chars + ypred
    test_chars = test_chars[1:] + ypred


Generating from seed: gry for it
gry for it me the was to the can alice the mure was to the me soin the was time so her to could ap to her and the wand a for, and the said the said the good was all of the dist the mowe were and the said was so the sith aping a dood to for the was of the dard the said the was to said the was with a me and the sing to her the pisting alice on to the mesterf the mare was tinn in it was and dot tee in but to s

<ipython-input-26-550eb7d147e8>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature
